**Course website**: http://lagex.github.io/geofisica2

**Note**: This notebook is part of the course "Geofísica 2" of Geology program of the 
[Universidade do Estado do Rio de Janeiro](http://www.uerj.br/). 
All content can be freely used and adapted under the terms of the 
[Creative Commons Attribution 4.0 International License](http://creativecommons.org/licenses/by/4.0/).

![Creative Commons License](https://i.creativecommons.org/l/by/4.0/88x31.png)

Esse documento que você está usando é um [IPython notebook](http://ipython.org/notebook.html). É um documento interativo que mistura texto (como esse), código (como abaixo), e o resultado de executar o código (que pode ser números, texto, figuras, videos, etc).

# Prática 8 - Sísmica de reflexão: wavelets, convolução e resolução



## Preparação

Rode a célula abaixo para carregar as componentes necessárias para fazer as simulações. Não se preocupe se aparecer um `:0: FutureWarning: IPython widgets are experimental and may change in the future.` abaixo. Isso é consequência de utilizar tecnologia de ponta.

In [ ]:
%matplotlib inline
from __future__ import division
import time
import urllib2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.html import widgets
from IPython import display
from fatiando import utils
from fatiando.seismic import conv
from fatiando.seismic.wavefd import Ricker
from fatiando.vis import mpl
import fatiando
plt.rc('font', size=10)

In [ ]:
print(fatiando.__version__)

## Convolução

A convolução é definida como:

$$
S(t) = R(t) \ast W(t) = \int\limits_{-\infty}^{+\infty} R(\tau)W(t - \tau)d\tau
$$

A reflexividade é:

$$
R = \frac{Amp_{Refletida}}{Amp_{Incidente}} = \frac{I_2 - I_1}{I_1 + I_2}
$$

$I = V \rho$ é a **impedância acústica**.

In [ ]:
tempos = np.linspace(0, 10, 201)
reflex = np.zeros_like(tempos)
reflex[55] = 0.5
reflex[138] = 1
def convolucao(t):    
    wave = Ricker(0.5, 1)
    sinal = np.zeros_like(tempos)
    for i, j in enumerate(tempos[tempos <= t]):
        w = wave(j - tempos)
        sinal[i] = (w*reflex).sum()
    fig, axes = plt.subplots(1, 3, figsize=(7, 6))
    for ax, data in zip(axes, [reflex, w, sinal]):
        ax.plot(data, tempos, '-k')
        ax.invert_yaxis()
        ax.set_xlim(-1., 1.)
        ax.set_xticks([-0.5, 0, 0.5])
        ax.grid()
        ax.hlines(t, -1, 1, colors='b', linestyles='--')
        ax.hlines(tempos[reflex > 0], -1, 1, colors='r', linestyles='--')
    for ax, label in zip(axes, [r"$\tau$", r"$t - \tau$", r"$t$"]):
        ax.set_ylabel(label, fontsize=18)
    for ax, title in zip(axes, [r"$R(\tau)$", r"$W(t - \tau)$", r"$S(t) = R(t) \ast W(t)$"]):
        ax.set_title(title, fontsize=16)
    plt.tight_layout()
widgets.interactive(convolucao, t=widgets.FloatSlider(min=0, max=tempos.max(), step=0.2, value=0))

## Resolução

$$
\lambda = \frac{V}{f} = V T
$$

Resolução máxima $\approx \lambda/4$.

No exemplo abaixo, $V_1 = 3000\ m/s$, $\rho_1 = 1600\ kg/m^3$, $\rho_2 = 1800\ kg/m^3$.

Leitura recomendada: [Tuning geology](http://www.agilegeoscience.com/blog/2011/7/8/tuning-geology.html)

In [ ]:
def make_wedge(n_samples, n_traces, layer_1_thickness, start_wedge, end_wedge):   
    rock_grid = np.zeros((n_samples, n_traces)) 
    for j in np.arange(n_traces):     
        for i in np.arange(n_samples):      
            if i <= layer_1_thickness:      
                rock_grid[i][j] = 1             
            if i > layer_1_thickness:         
                rock_grid[i][j] = 3                
                if j >= start_wedge and i - layer_1_thickness < j-start_wedge: 
                    rock_grid[i][j] = 2                    
                    if j >= end_wedge and i > layer_1_thickness+(end_wedge-start_wedge):
                        rock_grid[i][j] = 3
    return rock_grid

def resolucao(espessura, velocidade, frequencia, wedge):
    shape = (200, 100)
    n_samples, n_traces = shape 
    l1 = 60
    if wedge:
        rock_grid = make_wedge(n_samples, n_traces, l1, 5, espessura)
    else:
        rock_grid = np.ones(shape)
        rock_grid[l1:l1 + espessura] = 2
    vp = rock_grid.copy()
    vp[(rock_grid == 1) | (rock_grid == 3)] = 3000
    vp[rock_grid == 2] = velocidade
    rho = rock_grid.copy()
    rho[(rock_grid == 1) | (rock_grid == 3)] = 1600
    rho[rock_grid == 2] = 1800
    dz = 1000/200
    dt = 0.001
    vp_t, rho_t = conv.depth_2_time(vp.shape[0], vp.shape[1], vp, dt=dt, dz=dz, rho=rho)
    data = conv.seismic_convolutional_model(vp.shape[1], vp_t, frequencia, conv.rickerwave, dt=dt, rho=rho_t)
    times = np.arange(vp_t.shape[0])*dt
    profiles = np.array([15, 50, 90])
    plt.figure(figsize=(12, 4.5))
    ax = plt.subplot(121)
    mpl.seismic_image(data, dt=dt, cmap='RdBu_r')
    plt.vlines(profiles, 0, times.max(), colors='k', linestyles='--')
    plt.xlabel('x')
    plt.ylabel('tempo (s)')
    axes = [plt.subplot(185), plt.subplot(186), plt.subplot(187)]
    for ax, i in zip(axes, profiles):
        ax.plot(data[:, i], times, '-k')
        ax.set_ylim(0, times.max())
        ax.set_xlim(-0.5, 0.5)
        ax.invert_yaxis()
        ax.set_yticklabels([])        
        ax.set_xticks([-0.25, 0, 0.25])
        ax.grid()
    plt.tight_layout(pad=0)
widgets.interactive(resolucao, espessura=(20, 80, 5), velocidade=(3100, 5000, 100), frequencia=(4, 30, 2), wedge=True)

## Falha

In [ ]:
link = urllib2.urlopen('https://raw.githubusercontent.com/lagex/geofisica2/master/_static/img/model-falha.png')
with open('model-falha.png', 'wb') as f:
    f.write(link.read())

In [ ]:
vp = utils.fromimage('model-falha.png', (3000, 5000))
rho = vp/2.5

In [ ]:
def falha(frequencia):
    dt = 0.001
    data = conv.seismic_convolutional_model(vp.shape[1], vp, frequencia, conv.rickerwave, dt=dt, rho=rho)
    plt.figure(figsize=(14, 4))
    plt.suptitle(r'Camada fina  -  $h = 37.5\ m$  -  $\lambda/4 = {:.1f}\ m$'.format(0.25*5000/frequencia), 
                 fontsize=14)
    plt.subplot(1, 2, 1)
    mpl.seismic_image(data, dt=dt, cmap='RdBu_r', aspect='auto')
    plt.xlabel('x')
    plt.ylabel('tempo (s)')
    ax = plt.subplot(1, 2, 2)
    plt.imshow(vp, cmap='copper_r', aspect='auto')
    plt.colorbar(pad=0, aspect=30).set_label('velocidade (m/s)')
    ax.set_yticklabels([])
    plt.xlabel('x')
    plt.subplots_adjust(wspace=0.05)
widgets.interactive(falha, frequencia=(5, 50, 5))